In [1]:
from typing import Any

# The big python metaprograming

## Building blocks of code

- statements
- functions
- classes

### statements

- Perform the actual work
- Always execute in two scope
    - globals - Module dictionary
    - locals - Enclosing finctuons
- exec(statements [, globals [, locals]]) # Will see later?

### functions

- The fundamental unit of code
- The different calling convetnions
- Closure

#### The fundamental unit of code
    - Modul level functions
    - Methods of classes

#### The different calling convetnions
    - Positional arguments
    - Keyword arguments
        - Default arguments
            - These are set at definition time
            - Generaly it is a bad ide to use mutable values
                - Dont use lists or dictionaries as defaults
    - *args and **kwargs
        - collecting positional and keyword arguments
        - you can pass touples and dictionaries in a similar fashion
    - Keyword-Only Args
        - def recv(maxsize, *, block = True):
        - after the star you can only put keyword args
        - with this you can add keyword args to a function wich takes any number of positional        

#### Closure
    - You can make and return functions
    - Local variables are captured


In [2]:
def make_adder(x, y):

    def add():
        return x + y

    return add

In [3]:
a = make_adder(2, 3)
b = make_adder(10, 20)
print(a())
print(b())

5
30


### Classes

In [4]:
class Spam:
    a = 1

    def __init__(self, b):
        self.b = b

    def imethod(self):
        pass

    @classmethod
    def cmethod(cls):
        pass

    @staticmethod
    def smethod():
        pass

#### Short summary

- Variables
    - Class variable
    - Instance variable
- Methods
    - Class method
    - Static method
    - Instance method
    - Special methods
- Inheritance
- Dictionaries?


#### Variables

In [5]:
print(Spam.a)  # Class variable

1


In [6]:
s = Spam(2)
print(s.b)  # Instance variable

2


#### Methods

In [7]:
Spam.cmethod()  # Class method, execute it on the class

In [8]:
Spam.smethod()  # Static method, just a function in the class

In [9]:
s.imethod()  # Instance method only, execute it on the instance!

In [10]:
class Array:

    def __getitem__(self, index):
        pass

#### Inheritance

In [11]:
# Define a base calss
class Base:

    def spam(self):
        pass


# Inherite from that and make some changes
class Foo(Base):

    def spam(self):
        # Call method from base class using super
        r = super().spam()

#### Dictionaries

In [12]:
class Spam:

    def __init__(self, x, y):
        self.x = x
        self.y = y

    def foo(self):
        pass

In [13]:
# An instance is basically a dictionary
s = Spam(2, 3)

In [14]:
print(s.__dict__)

{'x': 2, 'y': 3}


In [15]:
print(Spam.__dict__['foo'])

<function Spam.foo at 0x7fb8c4be5c20>


## Metaprograming Basics


### The problem of debuging

#### Short summary


- the function with the print statement
- the problem with that it gets repetitive
- to solve this we introduce decorators
- so we have introduded new problems
- to solve this we wrap
- decorator with arguments
- can we debug all methods in a class?
- how deep can we debug methods in a class?
- debug all classes???
- what is a class?
- why to use a metaclass?
- big picture

#### Introduce decorators

In [16]:
def debug(func):

    def wrapper(*args, **kwargs):
        print(func.__name__)
        return func

    return wrapper

In [17]:
def add(x, y):
    return x + y


def sub(x, y):
    return x - y


def mul(x, y):
    return x * y


@debug
def div(x, y):
    return x / y

In [18]:
print(mul(2, 3))

6


In [19]:
print(div(2, 3))

div
<function div at 0x7fb8c5498b90>


#### We have introduced new problems

In [20]:
# We lose information on name
print(div)

<function debug.<locals>.wrapper at 0x7fb8c5498560>


In [21]:
# We lose information on helps
print(help(div))

Help on function wrapper in module __main__:

wrapper(*args, **kwargs)

None


#### To solve this we wrap

In [22]:
from functools import wraps
import logging
import os


def debug(func):
    if 'DEBUG' not in os.environ:
        return func  # you can optionally scip applying this decorator withe env variables
    log = logging.getLogger(func.__module__)  # That is a logging variation
    msg = func.__qualname__  #the fully qualified function name (What does this mean?)
    # It gets impartant with classes?
    # Claass name gets printed out with the method
    #@wraps(func) #Explicit way
    @wraps  # Copy metadata
    def wrapper(*args, **kwargs):
        print(msg)
        return func

    return wrapper


# Key idea to change the decarator independently form the code using it

In [23]:
sub = debug(sub)


# is equal to
@debug
def sub(x, y):
    return x - y

#### Decorator with arguments

In [24]:
from functools import wraps, partial
import logging
import os

os.environ["DEBUG"] = '1'


def debug(func=None, *, prefix=''):
    # This is how you make prefix variable available
    if func is None:  # It means that this was not pathed
        # How do we use partial?
        return partial(debug, prefix=prefix)
    #if 'DEBUG' not in os.environ:
    #    return func # you can optionally scip applying this decorator withe env variables
    log = logging.getLogger(func.__module__)  # That is a logging variation
    msg = prefix + func.__qualname__  #the fully qualified function name (What does this mean?)
    # It gets impartant with classes?
    # Claass name gets printed out with the method
    #@wraps might not work
    @wraps(func)  #Explicit way
    def wrapper(*args, **kwargs):
        print(msg)
        return func(*args, **kwargs)

    return wrapper


# Key idea to change the decarator independently form the code using it

In [25]:
@debug(prefix="*****")
def add(x, y):
    return x + y

In [26]:
print(add(2, 4))

*****add
6


#### Can we debug all methods in a class?

In [27]:
#Is this really what we looking for?
# This is repetitiv?
# What about class decorators?
class Spam:
    a = 1

    @debug
    def __init__(self, b):
        self.b = b

    @debug
    def imethod(self):
        pass

    @classmethod
    @debug
    def cmethod(cls):
        pass

    @staticmethod
    @debug
    def smethod():
        pass

In [28]:
def debugmethods(cls):
    #cls is class
    # Take a look, how we use that classes are just dictionaries with extras
    for name, val in vars(cls).items():
        if callable(val):
            setattr(cls, name, debug(val))
    return cls

In [29]:
# What about class decorators?
@debugmethods
class Spam:
    a = 1

    def __init__(self, b=3):
        self.b = b

    def imethod(self):
        pass

    @classmethod  # Not wrapped
    def cmethod(cls):
        pas

    @staticmethod  # Not wrapped
    def smethod():
        pass

In [30]:
s = Spam()

Spam.__init__


#### How deep can we debug methods in a class?

In [31]:
def debugatt(cls):
    # __getattribute__ lookup attrs from a class
    orig_getattribute = cls.__getattribute__

    # here we redefine the this functionality and
    def __getattribute__(self, name):

        # add somethong to it
        print("Get:", name)
        return orig_getattribute(self, name)

    # drop the new version to the class
    cls.__getattribute__ = __getattribute__

    return cls

In [32]:
@debugatt
class Point:

    def __init__(self, x, y):
        self.x = x
        self.y = y

In [33]:
p = Point(2, 3)

In [34]:
print(p.x)

Get: x
2


#### Debug all classes???

The solution for this is Metaclass

In [35]:
class debugmeta(type):

    def __new__(cls, clsname, bases, clsdict):
        # first we create a class normaly
        clsobj = super().__new__(cls, clsname, bases, clsdict)
        # later we apply a class decorator to it
        # be careful, classmethods and static methods are not wrapped !
        clsobj = debugmethods(clsobj)
        return clsobj

In [36]:
class Base(metaclass=debugmeta):
    pass

In [37]:
class Spam(Base):
    pass

In [38]:
a = Base()

In [39]:
b = Spam()

#### What is a class? Deconstructing class:

- in python every value has a type
- every type is defined by a class
- types of classes are all instances of types
- classes are instances of tpyes 
- so what is type?
    - tpye must be a class, cause classes makes instances!
    - and it makes instances of type

In [40]:
class Base:
    pass


class Spam(Base):

    def __init__(self, name):
        self.name = name

    def bar(self):
        print("I am Spam")

- What do we have?
    - Name
    - Base classes
    - Functions

- What happens during class definition?
    - The body of the fn gets isolated

##### Step 1

Body of the class gets isolated

In [41]:
body = '''def __init__(self, name):
    self.name = name
def bar(self):
    print("I am Spam")
'''

- The interpreter will make a dictionary that will serve as the class name space

##### Step 2

The interpreter will make a dictionary, class dictionary is created, that serves as the classe's namesapce. This is going to be "just a dictionary"

In [42]:
clsdict = type.__prepare__('Spam', (Base, ))

In [43]:
# As we can see its "just" dict
print(type(clsdict))

<class 'dict'>


##### Step 3

Body is executed in this dict

In [44]:
exec(body, globals(), clsdict)

In [45]:
# afterwards, clsdict is populated
print(clsdict)

{'__init__': <function __init__ at 0x7fb8c5498710>, 'bar': <function bar at 0x7fb8c5498d40>}


##### Step 4

Class is constructed from its name, base classes and the dictionary

In [46]:
# With this tpye() we are makeing the class
Spam = type('Spam', (Base, ), clsdict)

In [47]:
s = Spam('Guido')

In [48]:
s.bar()

I am Spam


We can change the metaclass

##### Abominations

Probably makeing a different type of type

In [49]:
# You can change the 'type' here for something
# TO 'ANYTHING'
class Spam(metaclass=type):

    def __init__(self, name):
        self.name = name

    def bar(self):
        print("I am Spam")

In [50]:
# You tipically inherit from tpye and redefine
class mytpye(type):

    def __new__(cls, clsname, bases, clsdict):
        # If the number of base classes is > 1
        if len(bases) > 1:
            raise TypeError("No!")
        return super().__new__(cls, clsname, bases, clsdict)

- And it works like

In [51]:
class Base(metaclass=mytpye):
    pass

In [52]:
class A(Base):
    pass

In [53]:
class B(Base):
    pass

In [54]:
# So if iy would print:

#class C(A, B):
#    pass

# I would get
"""
---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
/tmp/ipykernel_25442/1927587182.py in <module>
----> 1 class C(A, B):
      2     pass

/tmp/ipykernel_25442/2866177849.py in __new__(cls, clsname, bases, clsdict)
      3         # If the number of base classes is > 1
      4         if len(bases) > 1:
----> 5             raise TypeError("No!")
      6         return super().__new__(cls, clsname, bases, clsdict)

TypeError: No!

"""

'\n---------------------------------------------------------------------------\nTypeError                                 Traceback (most recent call last)\n/tmp/ipykernel_25442/1927587182.py in <module>\n----> 1 class C(A, B):\n      2     pass\n\n/tmp/ipykernel_25442/2866177849.py in __new__(cls, clsname, bases, clsdict)\n      3         # If the number of base classes is > 1\n      4         if len(bases) > 1:\n----> 5             raise TypeError("No!")\n      6         return super().__new__(cls, clsname, bases, clsdict)\n\nTypeError: No!\n\n'

In [55]:
# Do metaclasses have to inherete from type?
    # - Likely...  
# Very advanced question:
# What are problems that you can solve with a metaclass can not solve with a class decorator or the other way around

- Metaclasses get information about class definitions at the time of definition
    - can inspect this data
    - can midify this data
- They are similar to class decorators
- Python will execute any statement you put into a class body

#### Why to use a metaclass?

- They propagate down hierarchies

In [56]:
# Revisiting
def debugmethods(cls):
    #cls is class
    for name, val in vars(cls).items():
        if callable(val):
            setattr(cls, name, debug(val))
    return cls

# It gets inhereted!!!!!!!!!!!
class debugmeta(type):

    def __new__(cls, clsname, bases, clsdict):
        classobj = super().__new__(cls, clsname, bases, clsdict)
        # It gets inhereted!
        classobj = debugmethods(classobj)
        return classobj

In [57]:
# So you debug EEEEVRITHING
class Base(metaclass=debugmeta):
    pass

#### Big Picture

- Decorator wraps function
- Class Decorator wraps class
- Metaclass wraps Class hierarchies
- Order fo things
    - With metaclasses you can work before the class is created
    - With class decorators you can just work AFTER they are created

### The problem of Structures

#### Introduction

In [58]:
class Stock:

    def __init__(self, name, shares, price):
        self.name = name
        self.shares = shares
        self.price = price


class Point:

    def __init__(self, x, y):
        self.x = y
        self.y = y


class Address:

    def __init__(self, hostname, port):
        self.hostname = hostname
        self.port = port

In [59]:
class Structure:
    _fields = []
    def __init__(self, *args):
        # zip(self.__class__._fields, # would also work
        for name, val in zip(self._fields, args):
            setattr(self, name, val)

In [60]:
class Stock(Structure):
    _fields = ['name', 'shares', 'price']

class Point(Structure):
     _fields = ['x', 'y']


class Address(Structure):
     _fields = ['hostname', 'port']

In [61]:
# You can access a class variable through an instance

In [62]:
my_stock = Stock('gog', 1000, 17)

In [63]:
print(my_stock.name)

gog


- We will loose informacion on argument signiture
- We dont check the number of arguments while we create
- loose the ability to do key word arguments  

In [64]:
import inspect

In [65]:
print(inspect.signature(Stock))

(*args)


#### Signatures

In [66]:
from inspect import Parameter, Signature

In [67]:
_fields = ['name', 'shares', 'price']

In [68]:
parms = [ Parameter(fname, Parameter.POSITIONAL_OR_KEYWORD) for fname in _fields]

In [69]:
print(parms)

[<Parameter "name">, <Parameter "shares">, <Parameter "price">]


In [70]:
sig = Signature(parms)

In [71]:
print(type(sig))

<class 'inspect.Signature'>


In [72]:
# What can you do with this signature?

#### Binding Signatures

In [73]:
# Not great pattern, that we are using here an out of function scope object namely: sig)
def func(*args, **kwargs):
    # Sig.bind(binds positional / keyword to singnature)
    bound_args = sig.bind(*args, **kwargs)
    # .arguments is an OrderedDict of passed values
    for name, val in bound_args.arguments.items():
        print(name, "=", val)

In [74]:
func(1,2,3)

name = 1
shares = 2
price = 3


In [75]:
# bind can support keyword args
func(1,price=2,shares = 3)
# take a look on tthe print order!!

name = 1
shares = 3
price = 2


In [76]:
# bind chack arguments
try: 
    func(1,price=2)
except TypeError:
    print("TypeError: missing a required argument: 'shares'")

TypeError: missing a required argument: 'shares'


In [77]:
try: 
    # It is a bit tricky to catch a sintax error :) 
    eval('func(1,3, 2, 3)')
except TypeError:
    print("TypeError: too many positional arguments")

TypeError: too many positional arguments


In [78]:
from inspect import Parameter, Signature

In [79]:
def make_signature(names):
    return Signature(
        Parameter(
            name,
            Parameter.POSITIONAL_OR_KEYWORD) for name in names
        )

In [80]:
class Structure:
    __signature__ = make_signature([])
    def __init__(self, *args, **kwargs):
        # We are enforcing the signiture
        bound = self.__signature__.bind(*args, **kwargs)
        for name, val in bound.arguments.items():
            setattr(self, name, val)

In [81]:
class Stock(Structure):
    __signature__  = make_signature(['name', 'shares', 'price'])

class Point(Structure):
    __signature__  = make_signature(['x', 'y'])


class Address(Structure):
     __signature__  = make_signature(['hostname', 'port'])

In [82]:
s = Stock("Gog", 210, 43.1)

In [83]:
print(s.name)

Gog


In [84]:
print(s.shares)

210


In [85]:
p = Stock("Gor", shares = 10, price = 1)

Get: __class__
Get: __class__
Get: __class__
Get: __class__


In [86]:
print(p.name)

Gor


In [87]:
print(inspect.signature(Stock))

(name, shares, price)


#### Binding Signatures simplifyable?

- Pattern:
    - Problems involving class definitions are solved by
        - Class decorators
        - Meta classes
    - What seems better here?

##### With class decorators

In [88]:
def add_signature(*names):
    def decorate(cls):
        cls.__signature__ = make_signature(names)
        return cls
    return decorate

In [89]:
@add_signature("name", "shares", "price")
class Stock(Structure):
    pass

@add_signature("x", "y")
class Point(Structure):
    pass



In [90]:
t = Point(2, 3)

In [91]:
print(t.y)

3


##### With class decorators

In [92]:
class StructMeta(type):
    def __new__(cls, name, bases, clsdict):
        # Make the class
        clsobj = super().__new__(cls, name, bases, clsdict)
        # After makeing the class make the signatere
        sig = make_signature(clsobj._fields)
        # "drop" it on the class
        setattr(clsobj, '__signature__', sig)
        return clsobj

class Structure(metaclass = StructMeta):
    _fields = []
    def __init__(self, *args, **kwargs):
        bound = self.__signature__.bind(*args, **kwargs)
        for name, val in bound.arguments.items():
            setattr(self, name, val)

In [93]:
class Stock(Structure):
    _fields = ["name", "shares", "price"]

    
class Point(Structure):
     _fields = ["x", "y"]

In [94]:
z = Point(4, 6)

In [95]:
print(z.x)

4


In [96]:
z3 = Point(y = 3, x = 2)

In [97]:
print(z3.y)

3


##### Advanced like tyep check

In [98]:
#advanced like typcheck
class Structure(metaclass = StructMeta):
    _fields = []
    def __init__(self, *args, **kwargs):
        bound = self.__signature__.bind(*args, **kwargs)
        for name, val in bound.arguments.items():
            setattr(self, name, val)
    def __repr__(self):
        args = ', '.join(repr(getattr(self, name)) for name in self._fields)
        return type(self).__name__ + '(' + args + ')'

In [99]:
class Stock(Structure):
    _fields = ["name", "shares", "price"]

In [100]:
s = Stock("gagarin", 2, 3)

In [101]:
print(isinstance(s, Structure))

True


In [102]:
print(s.__repr__())

Stock('gagarin', 2, 3)


- Help functions does not loog for signatures 
- Does @wraps() propagate or does not propagate __signatures__ ?
- Can .bind(*args, ** kwargs) pushed to the metaclass?
    - no because bind is executed at instance creation
    - metaclass would be more about class definicion time

### Owning the dot

#### The problem of correctness

In [103]:
s = Stock('ACM', 30, 91.1)

In [104]:
s.name = 43
s.shares = "A lot"
s.price = (2323.22 + 2j)
## AND THIS IS A PROBLEM
# Where dowe validate?

In [105]:
class Stock(Structure):
    _fields = ["name", "shares", "price"]
    
    @property # getter
    def shares(self):
        return self._shares
    
    @shares.setter # setter
    def shares(self, value):
        if not isinstance(value, int):
            raise TypeError('expected int')
        if value < 0:
            raise ValueError('Must be >= 0')
        self._shares = value

In [106]:
s = Stock('ACM', 30, 11)

In [107]:
try:
    s.shares = "A lot"
except TypeError as R:
    print(R)

expected int


#### How to simplify this correctness

- Things we do:
    - Type check 
    - Domain / value checking
- Properties are implemmented via descriptors

In [108]:
# This "owns" the dot for a single atribute
class Descriptor:
    def __init__(self, name = None):
        # This is the name of the attribute being stored 
        self.name = name
        
    # If you use the same name in instance dictionary, the __get__ method is optional
    def __get__(self, instance, cls):
        # here the instance IS the instance we are working with
        print("Get", self.name)
        if instance is None:
            return self
        # Dierct manipulation of the instance dictionary
        return instance.__dict__[self.name]
    
    def __set__(self, instance, value):
        print("Set", self.name, value)
        # We can just drop a value on name?
        # Dierct manipulation of the instance dictionary
        instance.__dict__[self.name] = value
    
    def __delete__(self, instance):
        print("Delete", self.name)
        # Dierct manipulation of the instance dictionary
        del instance.__dict__[self.name]
        


In [109]:
def make_signature(names):
    return Signature(
        Parameter(
            name,
            Parameter.POSITIONAL_OR_KEYWORD) for name in names
        )

class StructMeta(type):
    def __new__(cls, name, bases, clsdict):
        # Make the class
        clsobj = super().__new__(cls, name, bases, clsdict)
        # After makeing the class make the signatere
        sig = make_signature(clsobj._fields)
        # "drop" it on the class
        setattr(clsobj, '__signature__', sig)
        return clsobj

class Structure(metaclass = StructMeta):
    _fields = []
    def __init__(self, *args, **kwargs):
        bound = self.__signature__.bind(*args, **kwargs)
        for name, val in bound.arguments.items():
            setattr(self, name, val)

In [110]:
class Stock(Structure):
    _fields = ["name", "shares", "price"]
    
    #shares will be captured by this  Descriptor object
    name = Descriptor('name')
    shares = Descriptor("shares")
    price = Descriptor('price')

In [111]:
s = Stock('ACM', 30, 11)
# >> Set shares 30

# Woow, we captured the event of setting shares attribuute to 30

Set name ACM
Set shares 30
Set price 11


In [112]:
print(s.shares)

Get shares
30


In [113]:
print(s.name)

Get name
ACM


In [114]:
del s.shares

Delete shares


In [115]:
s.shares = 50

Set shares 50


In [116]:
print(s.shares)

Get shares
50


In [117]:
print(s.price)

Get price
11


In [118]:
s.shares = 50
del s.shares

Set shares 50
Delete shares


#### Type Checking

In [119]:
# This "owns" the dot for a single atribute
class Descriptor:
    def __init__(self, name = None):
        # This is the name of the attribute being stored 
        self.name = name
        
    # If you use the same name in instance dictionary, the __get__ method is optional
    def __get__(self, instance, cls):
        # here the instance IS the instance we are working with
        print("Get", self.name)
        if instance is None:
            return self
        # Dierct manipulation of the instance dictionary
        return instance.__dict__[self.name]
    
    def __set__(self, instance, value):
        print("Set", self.name, value)
        # We can just drop a value on name?
        # Dierct manipulation of the instance dictionary
        instance.__dict__[self.name] = value
    
    def __delete__(self, instance):
        print("Delete", self.name)
        # Dierct manipulation of the instance dictionary
        del instance.__dict__[self.name]
        


In [120]:
class Typed(Descriptor):
    
    ty = object # The expected type
    # We are extending this descriptor
    def __set__(self, instance, value):
        if not isinstance(value, self.ty):
            raise TypeError('Expected %s' % self.ty) # Do this at least with fstring
        # Pass this for the __set__method
        super().__set__(instance, value)
        
class Integer(Typed):
    ty = int

class Float(Typed):
    ty = float
    
class String(Typed):
    ty = str
    
class Positive(Descriptor):
    def __set__(self, instance, value):
        if value < 0:
            raise ValueError('Must be >= 0')
        super().__set__(instance, value)

# We are combining two different implementations 
#  (i dont like this personally, cause it does not showes)
#  (that we are funneling, like combining functions, these classes are much more closer to functions and function combination)
# 
# Orther mathers here (just like f ° g =/= g ° f by default)
# you would do a numeric copareing on a non numeric type if you first check positiviness and you dont know if its an int or not
class PositiveInteger(Integer, Positive):
    
    pass

class PositiveFloat(Float, Positive):
    pass

######!!!!!!!!! Length checking (back to this after you got to MRO)
class Size(Descriptor):
    # # Keyword only argument pattern 
    def __init__(self, *args, maxlen, **kwargs):
        # We are just pulling a maxlen from the middle
        self.maxlen = maxlen
        super().__init__(*args, **kwargs)
        
    def __set__(self, instance, value):
        if len(value) >  self.maxlen:
            raise ValueError('Too BIG')
        super().__set__(instance, value)

class SizedString(String, Size):
    pass

# go full retard
import re

class Regex(Descriptor):
    # Keyword only argument pattern 
    def __init__(self, *args, pat, **kwargs):
        self.pat = re.compile(pat)
        super().__init__(*args, **kwargs)
        
    def __set__(self, instance, value):
        if not self.pat.match(value):
            raise ValueError('Invalid string')
        super().__set__(instance, value)

class SizedRegexString(SizedString, Regex):
    pass

In [121]:
def make_signature(names):
    return Signature(
        Parameter(
            name,
            Parameter.POSITIONAL_OR_KEYWORD) for name in names
        )

class StructMeta(type):
    def __new__(cls, name, bases, clsdict):
        # Make the class
        clsobj = super().__new__(cls, name, bases, clsdict)
        # After makeing the class make the signatere
        sig = make_signature(clsobj._fields)
        # "drop" it on the class
        setattr(clsobj, '__signature__', sig)
        return clsobj

class Structure(metaclass = StructMeta):
    _fields = []
    def __init__(self, *args, **kwargs):
        bound = self.__signature__.bind(*args, **kwargs)
        for name, val in bound.arguments.items():
            setattr(self, name, val)

In [122]:
class Stock(Structure):
    _fields = ["name", "shares", "price"]
    
    #shares will be captured by this  Descriptor object
    #name = String('name')
    # Two init functions, combined together
    name = SizedRegexString('name', pat = '[A-Z]+$', maxlen = 5)
    shares = PositiveInteger("shares")
    price = Float('price')

In [123]:
s = Stock("BARK", 3, 12.1)

Set name BARK
Set shares 3
Set price 12.1


In [124]:
s.name = "ACM"

Set name ACM


In [125]:
try: 
    s.name = 23
    # Interception is done before we hit the instance dictionary
    # The name of the descriptor and the name of the atrinute should be the same 
    #   Or we would need a __get__ method implemented 
except TypeError as R:
    print(R)

Expected <class 'str'>


In [126]:
try:
    s.shares = -1
except ValueError as R:
    print(R)

Must be >= 0


#### Understanding Method Resolution Order (MRO)

In [127]:
class PosInteger(Integer, Positive):
    pass

In [128]:
# This is regarding multiple inharitance
PosInteger.__mro__ # This chain defines the order in wich the value is checked by different __set__() methods
# This is the control flow of all those set functions
# Here the super of the "Typed" does not go to the parent "usually" ( it goes to the next thing on th MRO)
# The super of the positive will go to the base class
# with super you should understand diamond problems

(__main__.PosInteger,
 __main__.Integer,
 __main__.Typed,
 __main__.Positive,
 __main__.Descriptor,
 object)

In [129]:
try:
    s = Stock("BARKsds", 3, 12.1)
except ValueError as R:
    print(R)

Too BIG


In [130]:
try:
    s = Stock("Broo", 3, 12.1)
except ValueError as R:
    print(R)

Invalid string


#### Annoyance? of what?

In [131]:
class Stock(Structure):
    _fields = ["name", "shares", "price"]
    
    #shares will be captured by this  Descriptor object
    #name = String('name')
    # Two init functions, combined together
    name = SizedRegexString('name', pat = '[A-Z]+$', maxlen = 5)
    shares = PositiveInteger("shares")
    price = Float('price')

In [132]:
s = Stock("BONK", 4, 1.1)

Set name BONK
Set shares 4
Set price 1.1


### A new metaclass Annoyance? of what?

#### Getting to it with a new metaclass


In [133]:
from collections import OrderedDict

class StructMeta(type):
    @classmethod
    def __prepare__(cls, name, bases):
        """
        This is the class which returns the default empty dictionary
        which will then be filled and treated as the `__dict__` for a
        given class `cls`.
        We're overriding this and returning an ordered dictionary instead.
        This ensures that we keep the order of items for our arguments to
        `__init__` of `cls`. With unordered dictionary, the order of those
        arguments won't be preserved and *args might not have the intended
        order.
        """
        return OrderedDict()

    def __new__(cls, name, bases, clsdict):
        # This extracts the descriptor variable names
        # we could also use `val.name` instead, because that's
        # automatically obtained in descriptor with `__set_name__`
        fields = [
            key for key, val in clsdict.items()
            if isinstance(val, Descriptor)
        ]
        # Make class object itself, with normal dictionary, not an ordered one
        clsobj = super().__new__(cls, name, bases, dict(clsdict))
        # Produce the signature directly from obtained fields, rather than hard-coding
        # it in the class definition as `_fields`
        sig = make_signature(fields)
        setattr(clsobj, "__signature__", sig)
        return clsobj

In [134]:
class Structure(metaclass=StructMeta):
    """
    Automatically generate `__init__` which stores
    passed arguments accordingly to the names in
    `_fields`, which will automatically be used to generate
    a `__signature__` in the `StructMeta` defining meta class.
    """
    _fields = []

    def __init__(self, *args, **kwargs):
        bound = self.__signature__.bind(*args, **kwargs)
        for name, val in bound.arguments.items():
            setattr(self, name, val)

In [135]:
"""



from collections import OrderedDict
# If you dont oknow OrderedDict get to it
class StructMeta(type):
    # __Prepare is not in python 2...
    # When you execute the class body it executes it in a dictionary
    # It is mentioned that there are other hacks you can do to not use __prepare__
    @classmethod
    def __prepare__(cls, name, bases):
        # We will get rid of _fields by this trick
        return OrderedDict()
    
    
    def __new__(cls, name, bases, clsdict):
        # Not needing field because of this and keeping them ordered because of clsdict.items()
        # 'key' of this fields list is the name used in class dictionary
        # We really look just for descriptors
        # we are not interested in methods
        fields = [key for key, val in clsdict.items() if isinstance(val, Descriptor)]
        
        # We can fill in the name field 
        # because key above in fields is name used in calss dictionary
        for name in fields:
            clsdict[name].name = name
        
        # Make the class
        # with dict(clsdict) we need to create a proper dict for the class contents
        clsobj = super().__new__(cls, name, bases, dict(clsdict))
        
        # After makeing the class make the signatere
        sig = make_signature(fields)
        # "drop" it on the class
        setattr(clsobj, '__signature__', sig)
        return clsobj
"""

'\n\n\n\nfrom collections import OrderedDict\n# If you dont oknow OrderedDict get to it\nclass StructMeta(type):\n    # __Prepare is not in python 2...\n    # When you execute the class body it executes it in a dictionary\n    # It is mentioned that there are other hacks you can do to not use __prepare__\n    @classmethod\n    def __prepare__(cls, name, bases):\n        # We will get rid of _fields by this trick\n        return OrderedDict()\n    \n    \n    def __new__(cls, name, bases, clsdict):\n        # Not needing field because of this and keeping them ordered because of clsdict.items()\n        # \'key\' of this fields list is the name used in class dictionary\n        # We really look just for descriptors\n        # we are not interested in methods\n        fields = [key for key, val in clsdict.items() if isinstance(val, Descriptor)]\n        \n        # We can fill in the name field \n        # because key above in fields is name used in calss dictionary\n        for name in f

In [136]:
class Stock(Structure):
    # So because of __prepare__ we dont need fields attr
    # _fields = ["name", "shares", "price"]
    # It will record the order of the attributes based on how they are listed
    name = SizedRegexString(pat = '[A-Z]+$', maxlen = 5) # We got rid of 'name', because of 'keys'
    shares = PositiveInteger() # # We got rid of PositiveInteger('shares'), because of 'keys'
    price = Float()

In [137]:
s = Stock("BARK", 3, 12.1)

Set None BARK
Set None 3
Set None 12.1


#### Duplicate Definitions

In [138]:
class NoDupOrderedDict(OrderedDict):
    def __setitem__(self, key, value):
        if key in self:
            raise NameError('%s already defined' % key)
        super().__setitem__(key, value)
# And later use this on place of OrderedDict

In [139]:
class StructMeta(type):
    @classmethod
    def __prepare__(cls, name, bases):
        return NoDupOrderedDict()
    
    
    def __new__(cls, name, bases, clsdict):
        fields = [
            key for key, val in clsdict.items()
            if isinstance(val, Descriptor)
        ]
        clsobj = super().__new__(cls, name, bases, dict(clsdict))
        sig = make_signature(fields)
        setattr(clsobj, "__signature__", sig)
        return clsobj

In [140]:
class Structure(metaclass = StructMeta):
    _fields = []
    def __init__(self, *args, **kwargs):
        bound = self.__signature__.bind(*args, **kwargs)
        for name, val in bound.arguments.items():
            setattr(self, name, val)

In [141]:
try: 
    class Stock(Structure):
        name = String()
        shares = PositiveInteger()
        price = Float()
        shares = PositiveInteger()
except NameError as R:
    print(R)

shares already defined


- You can do multiple dispatch on method

### Performance in the bin with all these stuff?

- around 60-90 x hit on creation
    - mro chaseing...
    - the actual checking is minor part to it probably
- attribute lookup is uneffected (cause we did not implement __ get__ :P
- several bottlenecks
    - Signature enforcement
    - Multipy inheritacne / super in descriptor

#### Code generation

In [142]:
def _make_init(fields):
    """Generate full python code of an __init__ function"""
    code = f"def __init__(self, {', '.join(fields)}):\n"

    for name in fields:
        code += f"   self.{name} = {name}\n"

    return code

In [143]:
def _make_setter(dcls):
    """
    Takes descriptor class and produces the code for `__set__` function,
    to avoid super calls and slowdows from inheritance.
    It walks the method resolution order and collects the code from `set_code` function, 
    which it than concatenates to make a ne `__set__` method
    """
    code = "def __set__(self, instance, value):\n"
    for d in dcls.__mro__:
        if 'set_code' in d.__dict__:
            for line in d.set_code():
                code += f"   {line}\n"

    return code

In [144]:
class DescriptorMeta(type):
    """
    We need to use a metaclass for Descriptors, in order to automatically
    construct the `__set__` method for them using the code fragments
    from `set_code` method.
    """
    def __init__(self, clsname, bases, clsdict):
        """
        We're using `__init__` instead of `__new__`, because the
        generation of setter code using `_make_setter` requires
        us to have the MRO already estabolished, which is done
        when the class is being created in `__new__`, which has
        already happened in `__new__` so we can use `__init__`.
        """
        super().__init__(clsname, bases, clsdict)
        # In case somebody tries to implement the classical __set__
        # method, disallow it, it would be overridden anyway and the
        # use wouldn't know why, this way he'll be informed what's wrong
        if '__set__' in clsdict:
            raise TypeError("Use set_code(), not __set__()")
        # Make the `__set__` code
        code = _make_setter(self)
        exec(code, globals(), clsdict)
        # The actual __dict__ is altered and it becomes a mappingproxy
        # rather than normal dictionary and it will be read-only
        # the received `clsdict` is not necessarely the one that will be
        # used for the class itself, so we need to define the `__set__`
        # using setattr directly onto the class (`self`)
        setattr(self, "__set__", clsdict["__set__"])

In [145]:
class Descriptor(metaclass=DescriptorMeta):
    """
    This is a default descriptor implementation, it doesn't really do much
    but it provides us with a basis for all other descriptors, which can
    then override these methods (most notable setter, to impose some restrictions).
    """

    def __set_name__(self, owner_cls, name):
        """
        This method was implemented in python 3.6 (PEP 487), for any older versions
        you'll have to use meta-classes, to handle this automatically, or
        simply require the user to pass in the `name` in `__init__`.
        This implementation is shown in `extra_manual_name.py` file
        """
        self.name = name

    def __get__(self, instance, owner_cls):
        if instance is None:
            return self
        return instance.__dict__.get(self.name)

    @staticmethod
    def set_code():
        return ["instance.__dict__[self.name] = value"]

    def __delete__(self, instance):
        del instance.__dict__[self.name]

In [146]:
class Typed(Descriptor):
    """This is a general descriptor for enforcing types, it's expected to be subclassed."""
    ty = object  # Expected type

    @staticmethod
    def set_code():
        return [
            "if not isinstance(value, self.ty):",
            "   raise TypeError(f'Expected {self.ty.__name__}, got {type(value).__name__}.')",
        ]


In [147]:
class Integer(Typed):
    ty = int


class Float(Typed):
    ty = float

    @staticmethod
    def set_code():
        return [
            "if isinstance(value, int):",
            "   value = float(value)"
        ]


class String(Typed):
    ty = str


class Positive(Descriptor):
    @staticmethod
    def set_code():
        return [
            "if value < 0:",
            "   raise ValueError('Value must be positive (>= 0)')"
        ]


class PositiveInteger(Integer, Positive):
    pass


class PositiveFloat(Float, Positive):
    pass


class Regex(String):
    def __init__(self, *args, pattern, **kwargs):
        self.pattern = re.compile(pattern)
        return super().__init__(*args, **kwargs)

    @staticmethod
    def set_code():
        return [
            "if not self.pattern.match(value):",
            "   raise ValueError(\"String doesn't match the expected pattern\")"
        ]


class Sized(Descriptor):
    def __init__(self, *args, maxlen, **kwargs):
        self.maxlen = maxlen
        return super().__init__(*args, **kwargs)

    @staticmethod
    def set_code():
        return [
            "if len(value) > self.maxlen:",
            "   raise ValueError(f'Maximum expected length is {self.maxlen}, got {len(value)}')"
        ]


class SizedRegex(Regex, Sized):
    pass

In [148]:
class StructMeta(type):
    @classmethod
    def __prepare__(cls, name, bases):
        return NoDupOrderedDict()
    
    
    def __new__(cls, name, bases, clsdict):
        fields = [
            key for key, val in clsdict.items()
            if isinstance(val, Descriptor)
        ]
        if fields:
            init_code = _make_init(fields)
            exec(init_code, globals(), clsdict)# Do we need globals() here? or not?
            
        clsobj = super().__new__(cls, name, bases, dict(clsdict))
        # because we are doing an init we can get rid of signature thing
        # sig = make_signature(fields)
        # setattr(clsobj, "__signature__", sig)
        return clsobj

In [149]:
class Structure(metaclass=StructMeta):
    """
    Automatically generate `__init__` which stores
    passed arguments accordingly to the names in
    `_fields`, which will automatically be used to generate
    a `__signature__` in the `StructMeta` defining meta class.
    """
    _fields = []

In [150]:
class Stock(Structure):
    name = SizedRegex(maxlen=8, pattern=r"[A-Z]+$")
    shares = PositiveInteger()
    price = PositiveFloat()

In [151]:
s = Stock("ACCM",43, 17.1)
#https://imgs.xkcd.com/comics/ballmer_peak.png

In [152]:
print(s.name)

ACCM


### Esoteric stuff from xml

In [153]:
from xml.etree.ElementTree import parse

In [155]:
def _struct_to_class(structure):
    name = structure.get("name")
    code = f"class {name}(Structure):\n"
    for field in structure.findall("field"):
        descriptor_type = field.get("type")
        options = [
            f"{key} = {val}" for key, val in field.items()
            if key != "type"
        ]
        name = field.text.strip()
        code += f"   {name} = {descriptor_type}({', '.join(options)})\n"
    return code

def _xml_to_code(filename):
    doc = parse(filename)
    code = ""
    for structure in doc.findall("structure"):
        clscode = _struct_to_class(structure)
        code += clscode
    return code


code = _xml_to_code("classes.xml")
exec(code)

In [156]:
print(code)

class Stock(Structure):
   name = SizedRegex(maxlen = 8, pattern = '[A-Z]+$')
   shares = PositiveInteger()
   price = PositiveFloat()
class Address(Structure):
   hostname = String()
   port = Integer()



In [157]:
a = Address("Budapest", 112)

In [158]:
print(a.hostname)

Budapest


## Moveing On